In [5]:
import pandas as pd
import numpy as np

In [20]:
# %load ../../../helpers.py
import datetime
import json

def dated_filename (fn, ext='.csv'):
    today = datetime.date.today()
    return '{}-{}{}'.format(fn, today, ext)

def load_json (fn):
    with open(fn, 'r') as myfile:
        data=myfile.read()
    return json.loads(data)


In [6]:
users = pd.read_csv(
    '../../data/IT.NET.USER.ZS/API_IT.NET.USER.ZS_DS2_en_csv_v2_10576990.csv',
)
users.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Aruba,ABW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,62.0,69.0,74.000000,78.9,83.78,88.661227,93.542454,97.170000,NaN,NaN
1,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,5.0,5.454545,5.9,7.00,8.260000,10.595726,11.447688,NaN,NaN
2,Angola,AGO,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,2.8,3.1,6.500000,8.9,10.20,12.400000,13.000000,14.339079,NaN,NaN
3,Albania,ALB,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,45.0,49.0,54.655959,57.2,60.10,63.252933,66.363445,71.847041,NaN,NaN
4,Andorra,AND,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,81.0,81.0,86.434425,94.0,95.90,96.910000,97.930637,98.871436,NaN,NaN


In [8]:
def latest_statistic (row):
    latest = row.dropna()[-1]
    try:
        return float(latest)
    except:
        return np.nan

users['latest'] = users.apply(latest_statistic, axis=1).dropna()

In [14]:
pops = pd.read_csv(
    '../../data/SP.POP.TOTL/API_SP.POP.TOTL_DS2_en_csv_v2_10576638.csv'
)
pops['latest'] = pops.apply(latest_statistic, axis=1).dropna()
merged = users.merge(pops, how='right', on='Country Code')
merged = merged.rename(columns={
    'latest_x': 'percent-online', 
    'latest_y': 'population',
    'Country Name_x': 'Country Name',
})
merged['people-online'] = merged.apply(lambda row: row['population']*row['percent-online'], axis=1)

In [22]:
merged[['Country Code', 'Country Name', 'people-online']].to_csv(dated_filename('internet-users-country'))